### EDA Automation Notebook
This notebook is a jupyter notebook template for all my EDA projects. This will contain various checks that I commonly use in my projects. Since this is an automation or a template, this does not contain any test data. It will contain the following checks as seen below:
- Step 1: Loading the data
- Step 2: Checking the basic data information
- Step 3: Data quality assessment
- Step 4: Summary statistics
- Step 5: Correlations and outliers
- Step 6: Answering questions

There will be another python file and this prints out the pdf report instead of the notebook as this is used for data exploration only and not for answering questions.

In [23]:
#Import packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from IPython.display import display

##### Step 1: Loading the data

In [30]:
def load_data(file_path, sheet_name = None, encoding = "utf-8"):
    """
    Load a dataset either from a CSV or Excel file from any path provide by the user.

    Parameters:
        file_path (str): Path to the file, can be absolute or relative.
        sheet_name (str or int, optional): Sheet name is used as index for files.
    
    Output:
        dataframe: Data from file is loaded as a dataframe. Loads 50 rows(head) of data for better checking.
    """
    if len(file_path) == 0:
        raise FileNotFoundError(f"File path cannot be blank.")
    #Error handling for files if file cannot be located
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found for {file_path}. Ensure file is inside the folder or input the whole path instead.")

    ext_type = os.path.splitext(file_path)[-1].lower()

    #File type handling for CSV and Excel files as well
    try:    
        if ext_type == ".csv":
            #Unicode decode error handling
            try:
                df = pd.read_csv(file_path, encoding = encoding)
            except UnicodeDecodeError:
                print("UTF-8 failed, trying ISO-8859-1...")
                df = pd.read_csv(file_path, encoding = "ISO-8859-1")
        
        elif ext_type in [".xls", "xlsx"]:
            df = pd.read_excel(file_path, sheet_name = sheet_name if sheet_name else 0)
        
        #Error raising for unsupported file types
        else:
            raise ValueError("Unsupported file type. This notebook can only load CSV or Excel files.")

    except Exception as exc:
        raise RuntimeError(f"Error reading file: {exc}")

    print(f"Data loaded successfully from {file_path}")
    display(df.head(50))
    return df

In [ ]:
#Interactive data loading for targeted file path loading
if __name__ == "__main__":
    file_path = input("Enter file path (ex. data.csv or C:/folder/data.csv): ").strip()
    sheet_name = input("Optional: Specify sheet name or press ENTER to load the file directly.")
    sheet_name = sheet_name if sheet_name else None

    df = load_data(file_path, sheet_name)

##### Step 2: Checking the basic data information

##### Step 3: Data quality assessment

##### Step 4: Summary statistics

##### Step 5: Correlations and outliers

##### Step 6: Answering questions